In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

from bs4 import BeautifulSoup # class for web scraping. It allows you to interact with HTML in a similar way to how you would interact with a web page using developer tools.
import requests

In [54]:
browser = webdriver.Edge("msedgedriver.exe")
browser.maximize_window()
browser.get("https://www.imdb.com/")

# access the web and accept cookies automatically: //*[@id="__next"]/div/div/div[2]/div/button[2]
try:
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/div/div[2]/div/button[2]'))
    ).click()
    # Clicar en desplegable //*[@id="imdbHeader-navDrawerOpen"]/span
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]/span'))
    ).click()
    
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside[1]/div/div[2]/div/div[1]/span/div/div/ul/a[2]/span'))
    ).click()
    # sort by duration
    browser.find_element(By.XPATH, '//*[@id="sort-by-selector"]/option[8]').click()
except TimeoutException:
    pass

element = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul'))
)

table_html = element.get_attribute('outerHTML')


soup = BeautifulSoup(table_html, "html.parser")
titles = [title.text for title in soup.find_all('h3', class_='ipc-title__text')]
# Remove numbers from titles
titles = [title.split('.')[1] for title in titles]
titles = [title[1:] for title in titles]

califications = [calification.text for calification in soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')]
# from califications  only whats left of "\xa0"
califications = [calification.split('\xa0')[0] for calification in califications]

years = [year.text for year in soup.find_all('span', class_='sc-b0691f29-8 ilsLEX cli-title-metadata-item')]
# from years only interested in what have 4 digits
years = [year for year in years if len(year) == 4]

durations = [duration.text for duration in soup.find_all('span', class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item")]
# from durations only interested if contains "m"
durations = [duration for duration in durations if 'm' in duration]

In [47]:
# Create a DataFrame with Title, Year, Duration and Calification
df = pd.DataFrame(list(zip(titles, years, durations, califications)), columns=['Title', 'Year', 'Duration', 'Calification'])
print(df)

                                      Title  Year Duration Calification
0                El moderno Sherlock Holmes  1924      45m          8,2
1                                  El chico  1921    1h 8m          8,2
2               El maquinista de La General  1926   1h 18m          8,1
3                       Antes del atardecer  2004   1h 20m          8,1
4                                 Toy Story  1995   1h 21m          8,3
..                                      ...   ...      ...          ...
167                          Shutter Island  2010   2h 19m          8,2
168                                       L  1997   2h 19m          8,2
169                          Eva al desnudo  1950   2h 20m          8,2
170                     El club de la lucha  1999   2h 20m          8,8
171  Hasta el último hombre (Hacksaw Ridge)  2016   2h 20m          8,1

[172 rows x 4 columns]


In [59]:
# Pinchar otra vez en menú
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]/span'))
    ).click()
# Pinchar en Calendario de lanzamientos
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside[1]/div/div[2]/div/div[1]/span/div/div/ul/a[1]/span'))
    ).click()
# Tabla de lanzamientos
element = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/div[3]/section/section'))
)


table_html = element.get_attribute('outerHTML')
soup = BeautifulSoup(table_html, "html.parser")

# Buscar cada tabla: //*[@id="__next"]/main/div/div[3]/section/section/article[1]

# create a dataframe with movie and release date
df2 = pd.DataFrame(columns=['Date', 'Title'])

tables = soup.find_all('article')
for table in tables:
    date = table.find('h3', class_='ipc-title__text').text
    titles = [title.text for title in table.find_all('a', class_='ipc-metadata-list-summary-item__t')]
    # Add to dataframe
    for title in titles:
        df2.loc[len(df2)] = [date, title]